# snds example


In [2]:
from metadata.ingestion.ometa.ometa_api import OpenMetadata
from metadata.generated.schema.entity.services.connections.metadata.openMetadataConnection import (OpenMetadataConnection, AuthProvider)
from metadata.generated.schema.security.client.openMetadataJWTClientConfig import OpenMetadataJWTClientConfig

In [3]:
from creds import om_admin_token
server_config = OpenMetadataConnection(
    hostPort="http://datacatalog.casd.local/api",
    authProvider=AuthProvider.openmetadata,
    securityConfig=OpenMetadataJWTClientConfig(
        jwtToken=om_admin_token,
    ),
)
metadata = OpenMetadata(server_config)

In [4]:
# if it returns true, it means the connection is success 
metadata.health_check()

True

In [5]:
from metadata.generated.schema.api.services.createDatabaseService import CreateDatabaseServiceRequest
from metadata.generated.schema.entity.services.connections.database.common.basicAuth import BasicAuth
from metadata.generated.schema.entity.services.connections.database.mysqlConnection import MysqlConnection
from metadata.generated.schema.entity.services.databaseService import (DatabaseConnection, DatabaseService, DatabaseServiceType,)

db_service = CreateDatabaseServiceRequest(
    name="CASD-G4H-service",
    serviceType=DatabaseServiceType.Mysql,
    connection=DatabaseConnection(
        config=MysqlConnection(
            username="db_login",
            authType=BasicAuth(password="changeMe"),
            hostPort="http://db_url:1234",
        )
    ),
)

# when we create an entity by using function `create_or_update`, it returns the created instance of the query
db_service_entity = metadata.create_or_update(data=db_service)

In [6]:
from metadata.generated.schema.api.data.createDatabase import CreateDatabaseRequest

db_entity_req = CreateDatabaseRequest(
    name="SNDS",
    service=db_service_entity.fullyQualifiedName,
)

db_entity = metadata.create_or_update(data=db_entity_req)

In [7]:
from metadata.generated.schema.api.data.createDatabaseSchema import CreateDatabaseSchemaRequest

create_schema_req = CreateDatabaseSchemaRequest(
    name="DCIR", 
    database=db_entity.fullyQualifiedName)

# the create request will return the fqn(fully qualified name) of the created schema
schema_entity = metadata.create_or_update(data=create_schema_req)

In [8]:
from metadata.generated.schema.api.data.createTable import CreateTableRequest
from metadata.generated.schema.entity.data.table import Column, DataType

table_a = CreateTableRequest(
    name="ER_BIO_F",
    databaseSchema=schema_entity.fullyQualifiedName,
    columns=[Column(name="BIO_ACT_QSN", dataType=DataType.NUMBER,description="Quantite affinée signée de biologie"),
             Column(name="FLX_DIS_DTD", dataType=DataType.DATE,description="Date de mise à disposition des données"),
             Column(name="ORG_CLE_NEW", dataType=DataType.STRING,description="Organisme de liquidation des prestations (après fusion)"),
             Column(name="ORG_CLE_NUM", dataType=DataType.STRING,description="Ancien organisme avant fusion (jusqu’au jour J de la fusion)")],
)

table_a_entity = metadata.create_or_update(data=table_a)

In [11]:
import pandas as pd

table_name_path="../data/snds_tables.csv"
col_name_path="../data/snds_vars.csv"

table_df=pd.read_csv(table_name_path)
col_df = pd.read_csv(col_name_path)

In [12]:
table_df.head()

,Produit,Table,Libelle,creation,suppression,dates_manquantes
0,CARTOGRAPHIE_PATHOLOGIES,CT_DEP_AAAA_GN,Table dépenses de la cartographie des patholog...,2012.0,NaN,NaN
1,CARTOGRAPHIE_PATHOLOGIES,CT_IDE_AAAA_GN,Table individus de la cartographie des patholo...,2012.0,NaN,NaN
2,CARTOGRAPHIE_PATHOLOGIES,CT_IND_AAAA_GN,Table pathologies de la cartographie des patho...,2012.0,NaN,NaN
3,Causes de décès,KI_CCI_R,Table des circonstances et de la cause initial...,2006.0,NaN,NaN
4,Causes de décès,KI_ECD_R,Table de l’ensemble des causes de décès,2006.0,NaN,NaN


In [13]:
col_df.head()

,table,var,format,description,nomenclature,creation,suppression,dates_manquantes
0,BE_IDE_R,IDE_ETA_NUM,string (9),Numéro Finess de l'Etablissement,-,NaN,NaN,NaN
1,BE_IDE_R,IDE_ETA_NU8,string (8),Numéro Finess de l'Etabt sans clé,-,NaN,NaN,NaN
2,BE_IDE_R,IDE_ETA_NOM,string (38),Raison Sociale Abrégée,-,NaN,NaN,NaN
3,BE_IDE_R,IDE_IDE_CPL,string (38),Complément d'identification,-,NaN,NaN,NaN
4,BE_IDE_R,IDE_VOI_NUM,string (4),Numéro dans la voie,-,NaN,NaN,NaN


In [14]:
target_tab_name = "ER_BIO_F"
er_bio_f=table_df[table_df["Table"]==target_tab_name]

In [15]:
er_bio_f.head()

,Produit,Table,Libelle,creation,suppression,dates_manquantes
7,DCIR,ER_BIO_F,Prestations affinées BIOLOGIE,2006.0,NaN,NaN


In [16]:
er_bio_f_cols = col_df[col_df["table"]==target_tab_name][['table','var','format','description']]
er_bio_f_cols.head(20)

,table,var,format,description
850,ER_BIO_F,BIO_ACT_QSN,number (4),Quantite affinée signée de biologie
851,ER_BIO_F,BIO_ORD_NUM,number (4),Numéro d'ordre de l'acte affiné de biologie
852,ER_BIO_F,BIO_PRS_IDE,number (5),Code acte affiné de biologie
853,ER_BIO_F,DCT_ORD_NUM,number (9),Numéro d'ordre du décompte dans l'organisme
854,ER_BIO_F,FLX_DIS_DTD,date (20),Date de mise à disposition des données
855,ER_BIO_F,FLX_EMT_NUM,number (4),Numéro d'émetteur du flux
856,ER_BIO_F,FLX_EMT_ORD,number (4),Numéro de séquence du flux
857,ER_BIO_F,FLX_EMT_TYP,number (3),Type d'émetteur
858,ER_BIO_F,FLX_TRT_DTD,date (20),Date d'entrée des données dans le système d'in...
859,ER_BIO_F,ORG_CLE_NEW,string (9),Organisme de liquidation des prestations (aprè...


In [18]:
def getColDetailsByTabName(table_name:str, col_df):
    er_bio_f_cols = col_df[col_df["table"]==target_tab_name][['table','var','format','description']]
    er_bio_f_list=er_bio_f_cols.to_dict(orient="records")
    return er_bio_f_list
    

tab_col_list=getColDetailsByTabName(target_tab_name, col_df)

for item in tab_col_list:
    print(item['table'])
    print(item['var'])
    print(item['format'])
    print(item['description'])

ER_BIO_F
BIO_ACT_QSN
number (4)
Quantite affinée signée de biologie
ER_BIO_F
BIO_ORD_NUM
number (4)
Numéro d'ordre de l'acte affiné de biologie
ER_BIO_F
BIO_PRS_IDE
number (5)
Code acte affiné de biologie
ER_BIO_F
DCT_ORD_NUM
number (9)
Numéro d'ordre du décompte dans l'organisme
ER_BIO_F
FLX_DIS_DTD
date (20)
Date de mise à disposition des données
ER_BIO_F
FLX_EMT_NUM
number (4)
Numéro d'émetteur du flux
ER_BIO_F
FLX_EMT_ORD
number (4)
Numéro de séquence du flux
ER_BIO_F
FLX_EMT_TYP
number (3)
Type d'émetteur
ER_BIO_F
FLX_TRT_DTD
date (20)
Date d'entrée des données dans le système d'information
ER_BIO_F
ORG_CLE_NEW
string (9)
Organisme de liquidation des prestations (après fusion)
ER_BIO_F
ORG_CLE_NUM
string (9)
Ancien organisme avant fusion (jusqu’au jour J de la fusion)
ER_BIO_F
PRS_ORD_NUM
number (5)
Numéro d'ordre de la prestation dans le décompte
ER_BIO_F
REM_TYP_AFF
number (1)
Type de remboursement affiné
ER_BIO_F
DCT_ORD_NUM
number (9)
Numéro d'ordre du décompte dans l'organism

In [ ]:
from metadata.generated.schema.api.data.createTable import CreateTableRequest
from metadata.generated.schema.entity.data.table import Column, DataType


def create_table(table_name:str):
    table_a = CreateTableRequest(
    name=table_name,
    databaseSchema=schema_entity.fullyQualifiedName,
    columns=[Column(name="BIO_ACT_QSN", dataType=DataType.NUMBER,description="Quantite affinée signée de biologie"),
             Column(name="FLX_DIS_DTD", dataType=DataType.DATE,description="Date de mise à disposition des données"),
             Column(name="ORG_CLE_NEW", dataType=DataType.STRING,description="Organisme de liquidation des prestations (après fusion)"),
             Column(name="ORG_CLE_NUM", dataType=DataType.STRING,description="Ancien organisme avant fusion (jusqu’au jour J de la fusion)")],
)

    table_a_entity = metadata.create_or_update(data=table_a)
    table_create =  CreateTableRequest(
    name=table_name,
    databaseSchema=schema_entity.fullyQualifiedName,
    columns=[Column(name="uid", dataType=DataType.NUMBER,description="id of the user"),
             Column(name="oid", dataType=DataType.BIGINT,description="id of the order"),
             Column(name="product_id", dataType=DataType.BIGINT,description="product id"),
             Column(name="age", dataType=DataType.BIGINT,description="age of the user"),],
)